# Spotify Listening Trends' Dataset
To build the spotify dataset we will need to: 
1. Group data to quarterly dates
2. Get all unique songs, and run that dataset through build_genre_dataset.py to get all genre_tags.
3. Merge genre_tags to the charts dataset on [['artist','song_name']]
4. Clean dataset
5. Group genre_tags (ie. 'Rap' and 'Hip-Hop' to 'Rap/Hip-Hop')
6. Create the merged spotify and ONS dataset

Finally we need to inspect and clean the dataset.

## 1. GROUP DATES TO QUARTERLY FORMAT

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import re

In [4]:
df = pd.read_csv('../data/weekly_gb_merged.csv')
df.head(3)

,chart_position,chart_date,song,performer,streams,spotify_uri,song_id,region,time_on_chart,instance,consecutive_weeks,previous_week,peak_position,worst_position,chart_url
0,138,2018-03-22,$$$ - with Matt Ox,XXXTENTACION,386090,65u1dHQyQyE4y4aN2eDmMF,$$$ - with Matt OxXXXTENTACION,gb,1,1.0,NaN,NaN,138,138,https://charts.spotify.com/charts/view/regiona...
1,194,2017-10-19,'Till I Collapse,Eminem,274404,6yr8GiTHWvFfi4o6Q5ebdT,'Till I CollapseEminem,gb,1,1.0,NaN,NaN,194,194,https://charts.spotify.com/charts/view/regiona...
2,161,2017-11-16,'Till I Collapse,Eminem,321403,6yr8GiTHWvFfi4o6Q5ebdT,'Till I CollapseEminem,gb,2,2.0,NaN,NaN,161,194,https://charts.spotify.com/charts/view/regiona...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53596 entries, 0 to 53595
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   chart_position     53596 non-null  int64  
 1   chart_date         53596 non-null  object 
 2   song               53596 non-null  object 
 3   performer          53596 non-null  object 
 4   streams            53596 non-null  int64  
 5   spotify_uri        53596 non-null  object 
 6   song_id            53596 non-null  object 
 7   region             53596 non-null  object 
 8   time_on_chart      53596 non-null  int64  
 9   instance           53596 non-null  float64
 10  consecutive_weeks  46785 non-null  float64
 11  previous_week      46785 non-null  float64
 12  peak_position      53596 non-null  int64  
 13  worst_position     53596 non-null  int64  
 14  chart_url          53596 non-null  object 
dtypes: float64(3), int64(5), object(7)
memory usage: 6.1+ MB


In [6]:
# Chart date datatype converted to datetime in order to merge data with Quarterly stats and extract quarter/year
df['chart_date'] = pd.to_datetime(df['chart_date'])
df['quarter'] = df['chart_date'].dt.to_period('Q').dt.strftime('%Y Q%q')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53596 entries, 0 to 53595
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   chart_position     53596 non-null  int64         
 1   chart_date         53596 non-null  datetime64[ns]
 2   song               53596 non-null  object        
 3   performer          53596 non-null  object        
 4   streams            53596 non-null  int64         
 5   spotify_uri        53596 non-null  object        
 6   song_id            53596 non-null  object        
 7   region             53596 non-null  object        
 8   time_on_chart      53596 non-null  int64         
 9   instance           53596 non-null  float64       
 10  consecutive_weeks  46785 non-null  float64       
 11  previous_week      46785 non-null  float64       
 12  peak_position      53596 non-null  int64         
 13  worst_position     53596 non-null  int64         
 14  chart_

In [9]:
# Renaming columns so format is consistant with ONS data 
df.rename(columns={'chart_position': 'Chart Position', 
                   'chart_date': 'Chart Date',
                   'song': 'Song Title',
                   'performer': 'Performer',
                   'streams': 'Streams',
                   'spotify_uri': 'Spotify URI',
                   'song_id': 'Song ID',
                   'region': 'Region',
                   'time_on_chart': 'Time On Chart',
                   'instance': 'Instance',
                   'consecutive_weeks': 'Consecutive Weeks',
                   'previous_week': 'Previous Week',
                   'peak_position': 'Peak Position',
                   'worst_position': 'Worst Position',
                   'chart_url': 'Chart URL',
                   'quarter': 'Quarter'
                  }, inplace=True)

### Below filtering the dataset in order to display 
#### 1. Grouping data by all relevant identifiers and quarter
#### 2. Sorting chronologically with biggest hits first in each quarter
#### 3. Formating quarter for readability

In [10]:
# 1:
quarterly_data = (
    df.groupby([
        'Song ID',          # unique identifier
        'Spotify URI',      # Spotify's unique URI
        'Song Title',           
        'Performer',        
        'Region',           
        'Quarter'           # already in period[Q-DEC] format
    ])
    .agg({
        'Streams': 'sum',               # sum of weekly streams
        'Chart Position': 'min',        # best (lowest) position
        'Peak Position': 'min',         # confirming peak position
        'Worst Position': 'max',        # worst performance
        'Time On Chart': 'max',         # total weeks on chart
        'Consecutive Weeks': 'max',     # longest streak
        'Instance': 'first',            # keep first instance
        'Previous Week': 'first',       # keep first previous week
        'Chart URL': 'first'            # keep first URL
    })
    .reset_index()
    .rename(columns={
        'Streams': 'Quarterly Streams',
        'Chart Position': 'Best Quarterly Position'
    })
)

# 2:
quarterly_data = quarterly_data.sort_values([
    'Quarter', 
    'Quarterly Streams'
], ascending=[True, False])

# 3:
quarterly_data['Quarter'] = quarterly_data['Quarter'].astype(str).str.replace('Q', ' Q')

quarterly_data.head()

,Song ID,Spotify URI,Song Title,Performer,Region,Quarter,Quarterly Streams,Best Quarterly Position,Peak Position,Worst Position,Time On Chart,Consecutive Weeks,Instance,Previous Week,Chart URL
407,All I Want for Christmas Is YouMariah Carey,0bYg9bo50gSsH3LtXe2SQn,All I Want for Christmas Is You,Mariah Carey,gb,2016 Q4,2414608,1,1,1,1,NaN,1.0,NaN,https://charts.spotify.com/charts/view/regiona...
6697,Rockabye (feat. Sean Paul & Anne-Marie)Clean B...,5knuzwU65gJK7IF5yJsuaW,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,gb,2016 Q4,2406621,2,2,2,1,NaN,1.0,NaN,https://charts.spotify.com/charts/view/regiona...
4351,Last ChristmasWham!,2FRnf9qhLbvw8fu4IBXx78,Last Christmas,Wham!,gb,2016 Q4,2345002,3,3,3,1,NaN,1.0,NaN,https://charts.spotify.com/charts/view/regiona...
2493,Fairytale of New York (feat. Kirsty MacColl)Th...,3VTNVsTTu05dmTsVFrmGpK,Fairytale of New York (feat. Kirsty MacColl),The Pogues,gb,2016 Q4,1879767,4,4,4,1,NaN,1.0,NaN,https://charts.spotify.com/charts/view/regiona...
7602,StarboyThe Weeknd,5aAx2yezTd8zXrkmtKl66Z,Starboy,The Weeknd,gb,2016 Q4,1797034,5,5,5,1,NaN,1.0,NaN,https://charts.spotify.com/charts/view/regiona...


In [ ]:
# quarterly_data.to_csv('spotify_quarterly_merged.csv')

## 2. UNIQUE SONGS DATASET
Run unique_tracks.csv through build_genre_dataset.py, to create final_genre_tags.csv that contains the artist, song_name and genre tag for all unqiue songs in the charts dataset.

In [12]:
# final_merged_dataset.csv is spotify_quarterly_merged.csv after Nikki's addition and Ambers cleanup
charts = pd.read_csv('final_merged_dataset.csv')
unique_tracks = charts[['artist', 'song_name']].drop_duplicates().reset_index(drop=True)
# unique_tracks.to_csv('unique_tracks.csv', index=False)

## 3. MERGE genre_tags AND CHARTS DATASET

In [806]:
with open('get_genretags/final_genre_tags.json', 'r') as file:
    final_genre_tags = json.load(file)

In [807]:
final_genre_tags_df = pd.DataFrame(final_genre_tags)

# Merge Spotify genres
final_charts = pd.merge(charts,final_genre_tags_df,left_on=['song_name', 'artist'],right_on=['song_name', 'artist'],how='inner')
final_charts

,artist,song_name,quarterly_date,n_streams,genre
0,50 Cent,In Da Club,2009 Q4,NaN,Hip-Hop
1,Eminem,Without Me,2009 Q2,NaN,Hip-Hop
2,Mary J. Blige,Family Affair,2009 Q1,NaN,R&B
3,Akon,Smack That,2009 Q1,NaN,R&B
4,USHER,Yeah! (feat. Lil Jon & Ludacris),2009 Q2,NaN,r&b
...,...,...,...,...,...
9904,a-ha,Take on Me,2022 Q1,336351.0,Synthpop
9905,Britney Spears,Toxic,2022 Q1,335925.0,Pop
9906,"Internet Money, Gunna, Don Toliver, NAV","Lemonade (feat. Gunna, Don Toliver & NAV)",2022 Q1,334594.0,None
9907,Avicii,The Nights,2022 Q1,333649.0,House


In [808]:
# final_charts.to_csv('spotify_dataset.csv', index=False)

In [809]:
print(final_charts.shape)
final_charts['genre'].value_counts().head(10)

(9909, 5)


genre
Pop            827
pop            767
grime          695
rap            606
Hip-Hop        477
uk drill       261
soft pop       228
R&B            226
Indie          213
melodic rap    205
Name: count, dtype: int64

## 4. DATA CLEANSING

In [810]:
# Make all lower-case to avoid case sensitivity 
final_charts['genre'] = final_charts['genre'].str.lower()

In [811]:
print(final_charts['genre'].nunique())
final_charts['genre'].value_counts().head(60)

320


genre
pop                    1594
grime                   730
rap                     710
hip-hop                 477
indie                   343
r&b                     308
uk drill                261
soft pop                228
melodic rap             205
electronic              200
christmas               198
house                   177
singer songwriter       164
alternative rock        159
trap                    158
edm                     153
afroswing               149
soul                    147
classic rock            135
indie pop               133
rock                    123
emo rap                 119
rnb                     103
dance                   102
musicals                 82
brooklyn drill           81
myspotigrambot           79
hip hop                  63
britpop                  62
swing                    61
tropical house           61
folk                     56
electropop               55
country                  55
k-pop                    52
drill         

In [812]:
# Before doing any genre modifications the filtering code was double checked
# Example of check done before doing replace
final_charts[final_charts['genre'].str.contains(r'.*latino.*', regex=True, na=False)]


,artist,song_name,quarterly_date,n_streams,genre
2452,EO,German,2018 Q2,18186138.0,trap latino
2974,EO,German,2018 Q3,12182992.0,trap latino
3670,EO,German,2018 Q4,1900694.0,trap latino
9162,"Dave, Burna Boy",Location (feat. Burna Boy),2021 Q4,3785265.0,trap latino
9686,"Dave, Burna Boy",Location (feat. Burna Boy),2022 Q1,3067342.0,trap latino


### Rename genres for consistency 

In [813]:
# Where overlap of genres, smaller genres are given priority

# Non-overlapping genres
final_charts.loc[final_charts['genre'].str.contains(r'.*country.*', regex=True, na=False), 'genre'] = 'country'
final_charts.loc[final_charts['genre'].str.contains(r'.*christmas.*', regex=True, na=False), 'genre'] = 'christmas'
final_charts.loc[final_charts['genre'].str.contains(r'.*hip.hop.*', regex=True, na=False), 'genre'] = 'hip-hop'
final_charts.loc[final_charts['genre'].str.contains(r'.*hyphy.*', regex=True, na=False), 'genre'] = 'hip-hop'
final_charts.loc[final_charts['genre'].str.contains(r'.*polo g.*', regex=True, na=False), 'genre'] = 'hip-hop'
final_charts.loc[final_charts['genre'].str.contains(r'.*post malone.*', regex=True, na=False), 'genre'] = 'hip-hop'
final_charts.loc[final_charts['genre'].str.contains(r'.*chris brown.*', regex=True, na=False), 'genre'] = 'hip-hop'

final_charts.loc[final_charts['genre'].str.contains(r'.*dance.*', regex=True, na=False), 'genre'] = 'dance'
final_charts.loc[final_charts['genre'].str.contains(r'.*disco.*', regex=True, na=False), 'genre'] = 'disco'
final_charts.loc[final_charts['genre'].str.contains(r'.*folk.*', regex=True, na=False), 'genre'] = 'folk'
final_charts.loc[final_charts['genre'].str.contains(r'.*funk.*', regex=True, na=False), 'genre'] = 'funk'
final_charts.loc[final_charts['genre'].str.contains(r'.*soul.*', regex=True, na=False), 'genre'] = 'soul'
final_charts.loc[final_charts['genre'].str.contains(r'.*motown.*', regex=True, na=False), 'genre'] = 'soul'
final_charts.loc[final_charts['genre'].str.contains(r'.*musical.*', regex=True, na=False), 'genre'] = 'musicals'
final_charts.loc[final_charts['genre'].str.contains(r'.*swing.*', regex=True, na=False), 'genre'] = 'swing'
final_charts.loc[final_charts['genre'].str.contains(r'.*drill.*', regex=True, na=False), 'genre'] = 'drill'
final_charts.loc[final_charts['genre'].str.contains(r'.*eurovision.*', regex=True, na=False), 'genre'] = 'pop'
final_charts.loc[final_charts['genre'].str.contains(r'.*ballad.*', regex=True, na=False), 'genre'] = 'pop'
final_charts.loc[final_charts['genre'].str.contains(r'.*party.*', regex=True, na=False), 'genre'] = 'pop'

final_charts.loc[final_charts['genre'].str.contains(r'.*moombahton.*', regex=True, na=False), 'genre'] = 'edm'
final_charts.loc[final_charts['genre'].str.contains(r'.*trance.*', regex=True, na=False), 'genre'] = 'edm'
final_charts.loc[final_charts['genre'].str.contains(r'.*d.b.*', regex=True, na=False), 'genre'] = 'edm'  # For d&b, drum and bass, dubstep
final_charts.loc[final_charts['genre'].str.contains(r'drum & bass', regex=True, na=False), 'genre'] = 'edm'
final_charts.loc[final_charts['genre'].str.contains(r'.*garage.*', regex=True, na=False), 'genre'] = 'edm'

# Overlapping genres
final_charts.loc[final_charts['genre'].str.contains(r'.*punk.*', regex=True, na=False), 'genre'] = 'punk'
final_charts.loc[final_charts['genre'].str.contains(r'.*metal.*', regex=True, na=False), 'genre'] = 'metal'
final_charts.loc[final_charts['genre'].str.contains(r'.*rock.*', regex=True, na=False), 'genre'] = 'rock'
final_charts.loc[final_charts['genre'].str.contains(r'.*emo.*', regex=True, na=False), 'genre'] = 'rock'

final_charts.loc[final_charts['genre'].str.contains(r'.*house.*', regex=True, na=False), 'genre'] = 'house'
final_charts.loc[final_charts['genre'].str.contains(r'.*electro.*|.*techno.*', regex=True, na=False), 'genre'] = 'electronic'

final_charts.loc[final_charts['genre'].str.contains(r'.*trap.*', regex=True, na=False), 'genre'] = 'trap'
final_charts.loc[final_charts['genre'].str.contains(r'.*rap.*', regex=True, na=False), 'genre'] = 'rap'
final_charts.loc[final_charts['genre'].str.contains(r'.*jazz.*', regex=True, na=False), 'genre'] = 'jazz'

final_charts.loc[final_charts['genre'].str.contains(r'.*afrobeat.*', regex=True, na=False), 'genre'] = 'afrobeats'
final_charts.loc[final_charts['genre'].str.contains(r'^(?!.*(afrobeats))r.b', regex=True, na=False), 'genre'] = 'r&b'

final_charts.loc[final_charts['genre'].str.contains(r'.*reggaeton.*|.*latin.*|.*dembow.*', regex=True, na=False), 'genre'] = 'latin'
final_charts.loc[final_charts['genre'].str.contains(r'.*reggae.*', regex=True, na=False), 'genre'] = 'reggae'

# Chill genre
final_charts.loc[final_charts['genre'].str.contains(r'lullaby', regex=True, na=False), 'genre'] = 'chill'
final_charts.loc[final_charts['genre'].str.contains(r'piano', regex=True, na=False), 'genre'] = 'chill'
final_charts.loc[final_charts['genre'].str.contains(r'.*harmon.*', regex=True, na=False), 'genre'] = 'chill'

# Very general ones last
final_charts.loc[final_charts['genre'].str.contains(r'^(?!.*(soft pop|k-pop)).*pop.*', regex=True, na=False), 'genre'] = 'pop'
final_charts.loc[final_charts['genre'].str.contains(r'.*indie.*|.*alt.*', regex=True, na=False), 'genre'] = 'indie'


/var/folders/83/82m44yjd46xg7s3_zrbnvk4m0000gn/T/ipykernel_56019/2427758104.py:45: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_charts.loc[final_charts['genre'].str.contains(r'^(?!.*(afrobeats))r.b', regex=True, na=False), 'genre'] = 'r&b'
/var/folders/83/82m44yjd46xg7s3_zrbnvk4m0000gn/T/ipykernel_56019/2427758104.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  final_charts.loc[final_charts['genre'].str.contains(r'^(?!.*(soft pop|k-pop)).*pop.*', regex=True, na=False), 'genre'] = 'pop'


### Sort individual genres by reassigning genretag to songs

In [814]:

# Genre: british (manually rename the ones with most counts, and just reassign to pop others)
final_charts.loc[final_charts['song_name'] == 'Lost Without You', 'genre'] = 'indie'
final_charts.loc[final_charts['song_name'] == 'Will We Talk?', 'genre'] = 'rock'
final_charts.loc[final_charts['song_name'] == 'White Christmas - Spotify Singles - Holiday, Recorded at Air Studios, London', 'genre'] = 'christmas'
final_charts.loc[final_charts['genre'].str.contains(r'.*british.*', regex=True, na=False), 'genre'] = 'pop'

# Genre: new wave
final_charts.loc[final_charts['song_name'] == 'Sweet Dreams (Are Made of This) - Remastered', 'genre'] = 'pop'
final_charts.loc[final_charts['genre'].str.contains(r'.*new wave.*', regex=True, na=False), 'genre'] = 'rock'

# Genre: flamenco / flamenco urbano
final_charts.loc[final_charts['song_name'] == 'Enemy (with JID) - from the series Arcane League of Legends', 'genre'] = 'pop'
final_charts.loc[final_charts['song_name'] == 'Drive (feat. Wes Nelson)', 'genre'] = 'electronic' 
final_charts.loc[final_charts['song_name'] == 'Overseas', 'genre'] = 'rap' 
final_charts.loc[final_charts['genre'].str.contains(r'.*flamenco.*', regex=True, na=False), 'genre'] = 'hip-hop'

# Genre: fip
final_charts.loc[final_charts['song_name'] == 'Sometimes', 'genre'] = 'rock'
final_charts.loc[final_charts['song_name'] == 'Come Over (feat. Popcaan)', 'genre'] = 'r&b'
final_charts.loc[final_charts['song_name'] == 'A-O-K', 'genre'] = 'pop'

# Genre: ...
final_charts.loc[final_charts['genre'].str.contains(r'\...', regex=True, na=False), 'genre'] = 'rock'

# Genre: neoperreo
final_charts.loc[final_charts['song_name'].str.contains(r'.*Way 2 Sexy.*', regex=True, na=False), 'genre'] = 'hip-hop'
final_charts.loc[final_charts['song_name'].str.contains(r"Money Calling", regex=True, na=False), 'genre'] = 'r&b'



In [815]:
# Check state of genretags
print(final_charts['genre'].nunique())
final_charts['genre'].value_counts().head(40)

186


genre
pop                  1983
rap                  1118
rock                  800
grime                 730
hip-hop               631
r&b                   416
drill                 391
indie                 384
house                 316
electronic            268
soft pop              228
edm                   217
swing                 210
christmas             201
dance                 185
soul                  184
singer songwriter     164
musicals               87
country                87
myspotigrambot         79
folk                   63
latin                  60
metal                  55
k-pop                  52
punk                   37
soundtrack             36
disco                  33
jazz                   25
afrobeats              25
2010s                  22
2020                   20
funk                   18
chill                  15
reggae                 12
comedy                 12
2021                    9
2020s                   8
2019                    8
acoust

In [816]:
# Songs with Chistmas in the title, could consider retagging
pattern = r'.*Christmas.*|.*Santa.*|.*Bell.*'
final_charts.loc[(final_charts['song_name'].str.contains(pattern, regex=True, na=False)) & (final_charts['genre'].isnull()), 'genre'] = 'christmas'


### Manage unknown genre tags

In [817]:
# Rename weird genre tags to unknown
final_charts.loc[final_charts['genre'].str.contains(r'.*\d.*', regex=True, na=False), 'genre'] = 'unknown' # Contains digits
final_charts.loc[final_charts['genre'].str.contains(r'.*myspotigrambot.*', regex=True, na=False), 'genre'] = 'unknown'
final_charts.loc[final_charts['genre'].str.contains(r'.*singer.*', regex=True, na=False), 'genre'] = 'unknown'
final_charts.loc[final_charts['genre'].isna(), 'genre'] = 'unknown'


In [818]:
# Check state of genretags
print(final_charts['genre'].nunique())
final_charts['genre'].value_counts().head(60)


168


genre
pop                    1983
rap                    1118
rock                    800
unknown                 732
grime                   730
hip-hop                 631
r&b                     416
drill                   391
indie                   384
house                   316
electronic              268
soft pop                228
christmas               218
edm                     217
swing                   210
dance                   185
soul                    184
musicals                 87
country                  87
folk                     63
latin                    60
metal                    55
k-pop                    52
punk                     37
soundtrack               36
disco                    33
jazz                     25
afrobeats                25
funk                     18
chill                    15
comedy                   12
reggae                   12
acoustic                  8
american                  5
cover                     5
dua lipa      

In [819]:
# Remove rare genre tags
rare_genres = final_charts['genre'].value_counts()[lambda x: x <= 5].index
final_charts.loc[final_charts['genre'].isin(rare_genres), 'genre'] = 'unknown'

In [820]:
# How genre tags look after clean up
print(final_charts['genre'].nunique())
final_charts['genre'].value_counts()


33


genre
pop           1983
rap           1118
unknown       1005
rock           800
grime          730
hip-hop        631
r&b            416
drill          391
indie          384
house          316
electronic     268
soft pop       228
christmas      218
edm            217
swing          210
dance          185
soul           184
country         87
musicals        87
folk            63
latin           60
metal           55
k-pop           52
punk            37
soundtrack      36
disco           33
afrobeats       25
jazz            25
funk            18
chill           15
reggae          12
comedy          12
acoustic         8
Name: count, dtype: int64

In [821]:
# Look at unknown genre
final_charts[final_charts['genre'].str.contains(r'unknown', regex=True, na=False)]['song_name'].value_counts().head(10)

song_name
Shotgun                                      15
Someone You Loved                            14
Budapest                                     13
Before You Go                                11
Bruises                                      11
Hold Me While You Wait                        9
I'll Be There                                 8
Lemonade (feat. Gunna, Don Toliver & NAV)     8
Señorita                                      8
Hold My Girl                                  7
Name: count, dtype: int64

In [822]:
# Set genres for top artists with most counts
final_charts.loc[final_charts['artist'] == 'Lewis Capaldi', 'genre'] = 'pop'
final_charts.loc[(final_charts['artist'] == 'Shawn Mendes') & (final_charts['genre'] == 'unknown'), 'genre'] = 'pop'
final_charts.loc[(final_charts['artist'] == 'The Weeknd') & (final_charts['genre'] == 'unknown'), 'genre'] = 'r&b'
final_charts.loc[(final_charts['artist'] == '21 Savage') & (final_charts['genre'] == 'unknown'), 'genre'] = 'hip-hop'
final_charts.loc[(final_charts['artist'] == 'Tyler, The Creator') & (final_charts['genre'] == 'unknown'), 'genre'] = 'r&b'
final_charts.loc[(final_charts['artist'] == 'Tom Walker') & (final_charts['genre'] == 'unknown'), 'genre'] = 'indie'
final_charts.loc[(final_charts['artist'] == 'Lil Baby') & (final_charts['genre'] == 'unknown'), 'genre'] = 'hip-hop'
final_charts.loc[(final_charts['artist'] == 'Justin Bieber') & (final_charts['genre'] == 'unknown'), 'genre'] = 'pop'
final_charts.loc[(final_charts['artist'] == 'Jess Glynne') & (final_charts['genre'] == 'unknown'), 'genre'] = 'pop'
final_charts.loc[(final_charts['artist'] == 'Kodak Black') & (final_charts['genre'] == 'unknown'), 'genre'] = 'hip-hop'
final_charts.loc[(final_charts['artist'] == '6ix9ine') & (final_charts['genre'] == 'unknown'), 'genre'] = 'rap'

# Set genres for the top 7 songs with most counts
final_charts.loc[final_charts['song_name'] == 'Shotgun', 'genre'] = 'rock'
final_charts.loc[final_charts['song_name'] == 'Señorita', 'genre'] = 'latin'
final_charts.loc[final_charts['song_name'] == "I'll Be There", 'genre'] = 'pop'
final_charts.loc[final_charts['song_name'] == 'ROXANNE', 'genre'] = 'rap'
final_charts.loc[final_charts['song_name'] == 'Hold My Girl', 'genre'] = 'rock'
final_charts.loc[final_charts['song_name'].str.contains(r"Lemonade ", regex=True, na=False), 'genre'] = 'hip-hop'
final_charts.loc[final_charts['song_name'].str.contains(r"Cheerleader ", regex=True, na=False), 'genre'] = 'electronic'



In [823]:
# Assign genre to unknown genre_tags using already used tags associated to the same artist
# Get rows where genre is 'unknown'
unknown_rows = final_charts[final_charts['genre'] == 'unknown']

# Loop through each row with unknown genre
for row_index, row in unknown_rows.iterrows():
    # Get genres of other songs of the same artist as the unknown_rows songs
    artist_genres_series = final_charts.loc[(final_charts['artist'] == f"{row['artist']}") & (final_charts['genre']!='unknown'),'genre']
    genre = 'unknown'
    
    try:
        # Get genre which appears more for that artist
        genre = list(artist_genres_series.value_counts().keys())[0]    
    except IndexError:
        pass

    # Assign genre
    final_charts.loc[row_index,'genre'] = genre

In [824]:
# Final check
final_charts['genre'] = final_charts['genre'].str.title()

print(final_charts['genre'].nunique())
final_charts['genre'].value_counts()

33


genre
Pop           2207
Rap           1173
Rock           862
Hip-Hop        749
Grime          741
R&B            463
Indie          414
Drill          396
Unknown        368
House          329
Electronic     289
Christmas      225
Edm            219
Dance          217
Swing          212
Soft Pop       204
Soul           197
Country         88
Musicals        87
Latin           68
Folk            63
Metal           55
K-Pop           52
Punk            42
Soundtrack      41
Disco           33
Afrobeats       25
Jazz            25
Funk            18
Chill           15
Reggae          12
Comedy          12
Acoustic         8
Name: count, dtype: int64

## 5. GROUP GENRE CATEGORIES

In [825]:
uniform_tags = pd.read_csv('/Users/yrinavera/Desktop/group4-project/data/GenreGroup_Lookup.csv', encoding='latin-1')
new_tags = pd.DataFrame([{'genre': 'Rap', 'Genre_Group': 'Rap/Hip-Hop'},
            {'genre': 'Edm', 'Genre_Group': 'Electronic'},
            {'genre': 'Punk', 'Genre_Group': 'Rock/Punk/Metal'},
            {'genre': 'Unknown', 'Genre_Group': 'Unknown'}
           ])
uniform_tags = pd.concat([uniform_tags, new_tags])
final_charts_grouped = pd.merge(final_charts, uniform_tags, on = 'genre', how="left")
final_charts_grouped

,artist,song_name,quarterly_date,n_streams,genre,Genre_Group
0,50 Cent,In Da Club,2009 Q4,NaN,Hip-Hop,Rap/Hip-Hop
1,Eminem,Without Me,2009 Q2,NaN,Hip-Hop,Rap/Hip-Hop
2,Mary J. Blige,Family Affair,2009 Q1,NaN,R&B,R&B
3,Akon,Smack That,2009 Q1,NaN,R&B,R&B
4,USHER,Yeah! (feat. Lil Jon & Ludacris),2009 Q2,NaN,R&B,R&B
...,...,...,...,...,...,...
9904,a-ha,Take on Me,2022 Q1,336351.0,Pop,Pop
9905,Britney Spears,Toxic,2022 Q1,335925.0,Pop,Pop
9906,"Internet Money, Gunna, Don Toliver, NAV","Lemonade (feat. Gunna, Don Toliver & NAV)",2022 Q1,334594.0,Hip-Hop,Rap/Hip-Hop
9907,Avicii,The Nights,2022 Q1,333649.0,House,Dance/House


In [826]:
# final_charts_grouped.to_csv('spotify_dataset_new.csv', index=False)

## 6. MERGE ONS DATA

In [863]:
target = pd.read_csv('/Users/yrinavera/Desktop/group4-project/data/ons_genre_including_genregroup.csv')
ons_data = pd.read_csv('/Users/yrinavera/Desktop/group4-project/data/ONS/ons_filtered.csv')
print(target['Genre_Group'].value_counts())
target

Genre_Group
Rap/Hip-Hop        231
Pop                202
Rock/Punk/Metal    181
Dance/House        139
Electronic         138
Other               97
Folk                50
Soul                45
Reggae              42
Alternative         41
Afrobeats           36
Latin               35
Country             34
Indie               32
Disco               21
Soundtrack          21
Musicals            20
Rock & Roll         16
Chill               15
K-Pop               15
R&B                 11
Jazz                11
Funk                10
Acoustic             9
Christmas            9
Swing                6
Cover                4
Name: count, dtype: int64


,year_quarter,genre,Genre_Group,streams,gdp_(£m),inflation_(%),hfce_(£m),unemployment_rate_(%),max_streams_per_genre,is_max_streams,max_stream_genre
0,2016 Q4,Alternative Rock,Alternative,1505802,589476,1.5,369850,4.7,1505802,True,Christmas
1,2016 Q4,Baroque Pop,Pop,472598,589476,1.5,369850,4.7,472598,True,Christmas
2,2016 Q4,Celtic Rock,Rock/Punk/Metal,1879767,589476,1.5,369850,4.7,1879767,True,Christmas
3,2016 Q4,Christmas,Christmas,17142568,589476,1.5,369850,4.7,17142568,True,Christmas
4,2016 Q4,Classic Rock,Rock/Punk/Metal,753723,589476,1.5,369850,4.7,753723,True,Christmas
...,...,...,...,...,...,...,...,...,...,...,...
1466,2022 Q1,Trap Latino,Rap/Hip-Hop,3067342,629149,5.5,377996,3.8,3067342,True,Indie
1467,2022 Q1,Tropical House,Dance/House,362545,629149,5.5,377996,3.8,362545,True,Indie
1468,2022 Q1,Uk Drill,Rap/Hip-Hop,33303534,629149,5.5,377996,3.8,33303534,True,Indie
1469,2022 Q1,Uk Garage,Electronic,1280292,629149,5.5,377996,3.8,1280292,True,Indie


In [861]:
spotify_data = final_charts_grouped.copy()
spotify_data.dropna(subset = 'n_streams', inplace = True)
ons_genre = spotify_data.merge(ons_data, left_on= 'quarterly_date', right_on= 'Year Quarter',how='left')
ons_genre.drop(columns = ['Unnamed: 0','Year Quarter','artist'], inplace = True)
ons_genre.rename(columns={"quarterly_date": "year_quarter",
                          "n_streams": "streams",
                          "Gross Domestic Product (£m)": "gdp_(£m)",
                          "Unemployment Rate (%)": "unemployment_rate_(%)",
                          "CPIH (%)": "inflation_(%)",
                          "Household Final Consumption Expenditure (£m)":"hfce_(£m)" 
                         }, inplace=True)
ons_genre = ons_genre[['year_quarter','genre','Genre_Group','streams','gdp_(£m)','inflation_(%)','hfce_(£m)','unemployment_rate_(%)']]
ons_genre

,year_quarter,genre,Genre_Group,streams,gdp_(£m),inflation_(%),hfce_(£m),unemployment_rate_(%)
0,2016 Q4,R&B,R&B,2414608.0,589476.0,1.5,369850.0,4.7
1,2016 Q4,Pop,Pop,2406621.0,589476.0,1.5,369850.0,4.7
2,2016 Q4,Pop,Pop,2345002.0,589476.0,1.5,369850.0,4.7
3,2016 Q4,Rock,Rock/Punk/Metal,1879767.0,589476.0,1.5,369850.0,4.7
4,2016 Q4,R&B,R&B,1797034.0,589476.0,1.5,369850.0,4.7
...,...,...,...,...,...,...,...,...
9834,2022 Q1,Pop,Pop,336351.0,629149.0,5.5,377996.0,3.8
9835,2022 Q1,Pop,Pop,335925.0,629149.0,5.5,377996.0,3.8
9836,2022 Q1,Hip-Hop,Rap/Hip-Hop,334594.0,629149.0,5.5,377996.0,3.8
9837,2022 Q1,House,Dance/House,333649.0,629149.0,5.5,377996.0,3.8


In [864]:
ons_genre.to_csv('ons_genre_including_genregroup_new.csv', index=False)

In [868]:
print(ons_genre['genre'].unique())
print(ons_genre['Genre_Group'].unique())

['R&B' 'Pop' 'Rock' 'Christmas' 'House' 'Hip-Hop' 'Swing' 'Rap'
 'Electronic' 'Indie' 'Edm' 'Disco' 'Metal' 'Soft Pop' 'Unknown' 'Jazz'
 'Dance' 'Soul' 'Reggae' 'Musicals' 'Soundtrack' 'Funk' 'Grime' 'Folk'
 'Latin' 'Chill' 'Punk' 'Country' 'Afrobeats' 'Comedy' 'K-Pop' 'Acoustic'
 'Drill']
['R&B' 'Pop' 'Rock/Punk/Metal' 'Christmas' 'Dance/House' 'Rap/Hip-Hop'
 'Swing' 'Electronic' 'Indie' 'Disco' 'Unknown' 'Jazz' 'Soul' 'Reggae'
 'Musicals' 'Soundtrack' 'Funk' 'Folk' 'Latin' 'Chill' 'Country'
 'Afrobeats' 'Other' 'K-Pop' 'Acoustic']
